# Recepie Review and User Feedback Dataset

# CRIPS - DM Methodology
 1. Bussiness Understanding.
 2. Data Understanding. 
 3. Data Preparation.
 4. Modelling. 
 5. Evaluation. 
 6. Deplotment.

#### 1.Bussiness Understanding


- Recepies names information.
- The recepies has ranking on the top 100 best.
- Internal user reputation score (might have different weights based on their reputation)
- Comments has unique ID (most import up-votes).
- qualify sentimen 1 to 5 / 0 denoting absence of rating

- Bussiness can develop recomender systems 
- Bussiness can develop behavior analysis 

#### 2.Data Understanding

#### 3.Data Preparation

#### 4.Modelling

#### 5.Evaluation

#### 6.Deployment